## This script shows the visualization of electricity, water and gas consumption using carpet plots

To get started, first run the 'Synchronize data' script

#### Imports and paths

In [ ]:
import os, sys
import pytz, time
import inspect
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

script_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# add the path to opengrid to sys.path
sys.path.append(os.path.join(script_dir, os.pardir, os.pardir))
from opengrid.library import config, plotting
c=config.Config()

# find tmpo
sys.path.append(c.get('tmpo', 'folder'))

try:
    if os.path.exists(c.get('tmpo', 'data')):
        path_to_tmpo_data = c.get('tmpo', 'data')
except:
    path_to_tmpo_data = None

from opengrid.library.houseprint import houseprint

%matplotlib inline
plt.rcParams['figure.figsize'] = 16,8

# path to data
path_to_data = c.get('data', 'folder')
if not os.path.exists(path_to_data):
    raise IOError("Provide your path to the data in your config.ini file.  This is a folder containing a 'zip' and 'csv' subfolder.")
else:
    path_to_fig = os.path.join(path_to_data, 'figures')
    if not os.path.isdir(path_to_fig): os.makedirs(path_to_fig)

### Loading meta data and user variables

In [ ]:
hp = houseprint.load_houseprint_from_file('new_houseprint.pkl')


end = pd.Timestamp(time.time(), unit='s')
start = end - pd.Timedelta('30 days')

##Plotting

In [ ]:
hp.init_tmpo(path_to_tmpo_data=path_to_tmpo_data)

### Water sensors

In [ ]:
water_sensors = hp.get_sensors(sensortype='water')
print "{} water sensors".format(len(water_sensors))

In [ ]:
for sensor in water_sensors:
    tscum = sensor.get_data(head=start, tail=end)
    ts = tscum.diff()*60
    if not ts.dropna().empty:
        plotting.carpet(ts, title=sensor.device.key, zlabel=r'Flow [l/hour]')
        plt.savefig(os.path.join(path_to_fig, 'carpet_water_'+sensor.device.key+'_tmpo_'+sensor.key), dpi=100)

### Gas sensors

In [ ]:
gas_sensors = hp.get_sensors(sensortype=('gas'))
print "{} gas sensors".format(len(gas_sensors))

In [ ]:
for sensor in gas_sensors:
    tscum = sensor.get_data(head=start, tail=end)
    ts = tscum.diff()*60
    if not ts.dropna().empty:
        plotting.carpet(ts, title=sensor.device.key, zlabel=r'Flow [l/hour]')
        plt.savefig(os.path.join(path_to_fig, 'carpet_gas_'+sensor.device.key+'_tmpo_'+sensor.key), dpi=100)

### Electricity sensors

In [ ]:
elec_sensors = hp.get_sensors(sensortype=('electricity'))
print "{} electricity sensors".format(len(elec_sensors))

In [ ]:
for sensor in elec_sensors:
    tscum = sensor.get_data(head=start, tail=end)
    ts = tscum.diff()*60
    if not ts.dropna().empty:
        plotting.carpet(ts, title=sensor.device.key, zlabel=r'Power [W]')
        plt.savefig(os.path.join(path_to_fig, 'carpet_elec_'+sensor.device.key+'_tmpo_'+sensor.key), dpi=100)